<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

En este tutorial, aprenderás tres enfoques para **manejar valores faltantes**. Luego compararás la efectividad de estos enfoques utilizando un conjunto de datos del mundo real.

# Introducción

Existen muchas razones por las que un conjunto de datos puede tener valores faltantes. Por ejemplo:
- Una casa de 2 habitaciones no tendrá un valor registrado para el tamaño de una tercera habitación.
- Un encuestado puede optar por no revelar su ingreso.

La mayoría de las librerías de aprendizaje automático (incluyendo scikit-learn) generan un error si intentas construir un modelo utilizando datos con valores faltantes. Por lo tanto, deberás elegir una de las estrategias que se presentan a continuación.

# Tres Enfoques

### 1) Una opción simple: eliminar columnas con valores faltantes

La opción más simple consiste en eliminar las columnas que contienen valores faltantes.

![tut2_approach1](https://storage.googleapis.com/kaggle-media/learn/images/Sax80za.png)

A menos que la mayoría de los valores en dichas columnas estén ausentes, con este enfoque el modelo pierde el acceso a una gran cantidad de información (¡potencialmente útil!).  
Como ejemplo extremo, imagina un conjunto de datos con 10,000 filas, donde una columna importante solo tiene un único valor faltante. ¡Este enfoque eliminaría la columna por completo!

### 2) Una mejor opción: imputación

La **imputación** consiste en rellenar los valores faltantes con algún número. Por ejemplo, podemos completar con el valor promedio de cada columna.

![tut2_approach2](https://storage.googleapis.com/kaggle-media/learn/images/4BpnlPA.png)

El valor imputado no será exactamente correcto en la mayoría de los casos, pero usualmente produce modelos más precisos que si se eliminara la columna completa.

### 3) Una extensión de la imputación

La imputación es el enfoque estándar y, por lo general, funciona bien. Sin embargo, los valores imputados pueden estar sistemáticamente por encima o por debajo de sus valores reales (los cuales no se recolectaron). O bien, las filas con valores faltantes pueden ser únicas en otros aspectos. En estos casos, el modelo puede generar mejores predicciones si tiene en cuenta qué valores estaban originalmente ausentes.

![tut3_approach3](https://storage.googleapis.com/kaggle-media/learn/images/UWOyg4a.png)

En este enfoque, imputamos los valores faltantes como antes. Además, para cada columna que contenía valores faltantes en el conjunto de datos original, añadimos una nueva columna que indica la ubicación de las entradas imputadas.

En algunos casos, esto mejora significativamente los resultados. En otros casos, no ayuda en absoluto.

# Ejemplo

En el siguiente ejemplo trabajaremos con el [conjunto de datos de viviendas de Melbourne](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home). Nuestro modelo utilizará información como el número de habitaciones y el tamaño del terreno para predecir el precio de la vivienda.

No nos centraremos en el paso de carga de datos. En su lugar, imaginaremos que ya tienes los datos de entrenamiento y validación disponibles en las variables `X_train`, `X_valid`, `y_train` y `y_valid`.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar los datos
data = pd.read_csv('/content/melb_data.csv')

# Seleccionar la variable objetivo
y = data.Price

# Para simplificar, utilizaremos únicamente predictores numéricos
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Dividir los datos en subconjuntos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

### Definimos la función para medir la calidad de cada enfoque

Definimos una función llamada `score_dataset()` para comparar los diferentes enfoques para tratar los valores faltantes. Esta función reporta el [error absoluto medio (MAE)](https://es.wikipedia.org/wiki/Error_absoluto_medio) utilizando un modelo de bosque aleatorio.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Puntaje del Enfoque 1 (Eliminar columnas con valores faltantes)

Como estamos trabajando con conjuntos de entrenamiento y validación, debemos asegurarnos de eliminar las mismas columnas en ambos `DataFrames`.

In [4]:
# Obtener los nombres de las columnas con valores faltantes
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Eliminar las columnas con valores faltantes en los datos de entrenamiento y validación
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE del Enfoque 1 (Eliminar columnas con valores faltantes):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))


MAE del Enfoque 1 (Eliminar columnas con valores faltantes):
183550.22137772635


### Puntaje del Enfoque 2 (Imputación)

A continuación, utilizamos [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) para reemplazar los valores faltantes con el valor medio (promedio) de cada columna.

Aunque es un método simple, rellenar con el valor medio generalmente ofrece buenos resultados (aunque esto puede variar según el conjunto de datos).  
Si bien los estadísticos han experimentado con métodos más complejos para determinar los valores imputados (como la **imputación por regresión**, por ejemplo), estas estrategias sofisticadas normalmente no aportan beneficios adicionales cuando los resultados se integran en modelos avanzados de aprendizaje automático.


In [5]:
from sklearn.impute import SimpleImputer

# Imputación
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# La imputación eliminó los nombres de las columnas; los restauramos
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE del Enfoque 2 (Imputación):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))


MAE del Enfoque 2 (Imputación):
178166.46269899711


Observamos que el **Enfoque 2** tiene un MAE menor que el **Enfoque 1**, por lo tanto, el **Enfoque 2** tuvo un mejor desempeño en este conjunto de datos.

### Puntaje del Enfoque 3 (Una extensión de la imputación)

A continuación, imputamos los valores faltantes, pero además registramos qué valores fueron imputados.

In [6]:
# Hacer una copia para evitar modificar los datos originales (durante la imputación)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Crear nuevas columnas que indiquen qué valores serán imputados
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputación
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# La imputación eliminó los nombres de las columnas; los restauramos
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE del Enfoque 3 (Una extensión de la imputación):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))


MAE del Enfoque 3 (Una extensión de la imputación):
178927.503183954


Como podemos ver, el **Enfoque 3** tuvo un rendimiento ligeramente inferior al del **Enfoque 2**.

### Entonces, ¿por qué la imputación tuvo mejor desempeño que eliminar las columnas?

El conjunto de datos de entrenamiento tiene 10,864 filas y 12 columnas, de las cuales tres contienen valores faltantes.  
En cada una de estas columnas, menos de la mitad de las entradas están ausentes. Por lo tanto, eliminar estas columnas implica descartar una gran cantidad de información útil, lo que explica por qué la imputación ofrece un mejor rendimiento.

In [7]:
# Dimensión del conjunto de entrenamiento (número de filas, número de columnas)
print(X_train.shape)

# Número de valores faltantes en cada columna del conjunto de entrenamiento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])


(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# Conclusión

Como es habitual, imputar los valores faltantes (en el **Enfoque 2** y el **Enfoque 3**) produjo mejores resultados en comparación con simplemente eliminar las columnas con valores faltantes (como en el **Enfoque 1**).